# Development of a Mobile App for Lottery Addiction
Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:
* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

We'll be using historical data coming from the national 6/49 lottery game in Canada. The dataset has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

### Core Functions
Our goal is to write code that can enable users to answer probability questions about playing the lottery.

The app will repeatedly calculate probabilities and combinations. We'll start by writing 2 (two) functions that:
a. calculates factorials; and
b. calculates combinations.

In [2]:
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    combinations = numerator / denominator
    return combinations

### One-ticket Probability
In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn.

If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

We'll start by building a function that calculates the probability of winning the big prize for any given ticket.

The function will follow the details below:
* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [3]:
def one_ticket_probability(six_num_list):
    for i in six_num_list:
        if len(six_num_list) != 6:
            return 'Error! Please provide only 6 numbers. Try Again!'
        elif 1 > i or i > 49:
            return 'Error! A number is either less than 1 or greater than 49. Try Again!'
        elif not isinstance(i, int):
            return 'Error! Only integers allowed. Try Again!'
    total_combinations = combinations(49, 6)
    successful_outcomes = 1
    p_success = (successful_outcomes / total_combinations) * 100
    return f'The probability of success is {p_success: .7f}%'

Let's test the function.

In [4]:
list_1 = [1, 45, 8, 12, 39, 26]
list_2 = [6, 43, 27, 38, 15, 4]

In [5]:
print(one_ticket_probability(list_1))
print(one_ticket_probability(list_2))

The probability of success is  0.0000072%
The probability of success is  0.0000072%


The probability of success for one set of 6 numbers is 0.0000072%.
Let's test for error checking.

In [6]:
list_3 = [13, 0, 25, 5, 36, 47]
list_4 = [8, 3.4, 9, 45, 33, 7]
list_5 = [6, 67, 34, 43, 22, 5]
list_6 = [6, 67, 34, 43, 22, 5, 9]
list_7 = [6, 67, 34, 43, 22]

In [7]:
print(one_ticket_probability(list_3))
print(one_ticket_probability(list_4))
print(one_ticket_probability(list_5))
print(one_ticket_probability(list_6))
print(one_ticket_probability(list_7))

Error! A number is either less than 1 or greater than 49. Try Again!
Error! Only integers allowed. Try Again!
Error! A number is either less than 1 or greater than 49. Try Again!
Error! Please provide only 6 numbers. Try Again!
Error! Please provide only 6 numbers. Try Again!


Error checking is accurate.

### Historical Data Check for Canada Lottery
We'd also like the app to allow users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

We'll begin by loading the dataset of historical data into a pandas dataframe and review it.

In [8]:
import warnings

warnings.simplefilter(action='ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('649.csv')
df.shape

(3665, 11)

In [10]:
df.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [11]:
df.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


### Function for Historical Data Check
In developing the app, we need to be aware of the following details:
* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list and serve as an input to our function.
* We will write a function that prints:
    - the number of times the combination selected occurred in the Canada data set; and
    - the probability of winning the big prize in the next drawing with that combination.

We'll write this function below.

We'll begin by writing a function that takes as input a row of the lottery dataframe and returns a set containing all the six winning numbers. For the first row, for instance, the function should return the set {3, 41, 11, 12, 43, 14}.

We'll apply this function to the dataframe to create a series of a set of winning numbers. We'll use this series of sets to determine if a list of numbers have featured as winning numbers in the past.

In [12]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winning_numbers = df.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

We'll now write a function to compare a given list of numbers to the series created above to determine if there are any matches (and the number of matches) of the list in the series. In the function, we'll also inform the user what their chances are of winning the lottery using their list of numbers.

In [13]:
def check_historical_occurence(number_list, set_series):
    set_list = set(number_list)
    value_count_occur = (winning_numbers == set_list).sum()

    total_combinations = combinations(49, 6)
    successful_outcomes = 1

    print(f'Your number list {number_list} has occurred as a winning set {value_count_occur} times in the past. {one_ticket_probability(number_list)} which means you have a {successful_outcomes} in {total_combinations:,.0f} chance of winning the lottery with this number list.')

Let's test the function with 2 number lists

In [14]:
check_historical_occurence(list_2, winning_numbers)

Your number list [6, 43, 27, 38, 15, 4] has occurred as a winning set 0 times in the past. The probability of success is  0.0000072% which means you have a 1 in 13,983,816 chance of winning the lottery with this number list.


In [15]:
list_8 = [3, 41, 11, 12, 43, 14]
check_historical_occurence(list_8, winning_numbers)

Your number list [3, 41, 11, 12, 43, 14] has occurred as a winning set 1 times in the past. The probability of success is  0.0000072% which means you have a 1 in 13,983,816 chance of winning the lottery with this number list.


### Multi-ticket Probability
Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. The purpose of this app is to help them better estimate their chances of winning.

We'll write a function that will allow the users to calculate the chances of winning for any number of different tickets, based on the following criteria:
- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- The function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [22]:
def multi_ticket_probability(tix):
   total_outcomes = combinations(49, 6)
   successful_outcomes = tix
   p_success = (successful_outcomes / total_outcomes) * 100
   print(f'You have a {p_success:.5f}% probability of winning the lottery with {tix} tickets.')

Let's test our function.

In [23]:
list_9 = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for ticket in list_9:
    multi_ticket_probability(ticket)

You have a 0.00001% probability of winning the lottery with 1 tickets.
You have a 0.00007% probability of winning the lottery with 10 tickets.
You have a 0.00072% probability of winning the lottery with 100 tickets.
You have a 0.07151% probability of winning the lottery with 10000 tickets.
You have a 7.15112% probability of winning the lottery with 1000000 tickets.
You have a 50.00000% probability of winning the lottery with 6991908 tickets.
You have a 100.00000% probability of winning the lottery with 13983816 tickets.


### Less Winning Numbers — Function
We'll write a final function to allow users calculate probabilities for two, three, four or five winning numbers. This is because in 6/49 lotteries, there are smaller prizes for players whose tickets match two, three, four or five of the six numbers drawn.

The function will accept an integer between 2 and 5, that represents the number of winning numbers expected. The function will then print out information about the probability of having the inputted number of winning numbers.

In [27]:
def probability_less_6(int_2_5):
    n_combinations_ticket = combinations(6, int_2_5)
    n_combinations_remaining = combinations(43, 6 - int_2_5)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining

    total_outcomes = combinations(49, 6)
    p_success = (successful_outcomes / total_outcomes) * 100

    print(f'Your chances of having {int_2_5} winning numbers with this ticket are {p_success:.6f}%, i.e. a 1 in {round(total_outcomes / successful_outcomes):,} chance.')


Let's test the function.

In [28]:
list_10 = [2, 3, 4, 5]
for ticket in list_10:
    probability_less_6(ticket)

Your chances of having 2 winning numbers with this ticket are 13.237803%, i.e. a 1 in 8 chance.
Your chances of having 3 winning numbers with this ticket are 1.765040%, i.e. a 1 in 57 chance.
Your chances of having 4 winning numbers with this ticket are 0.096862%, i.e. a 1 in 1,032 chance.
Your chances of having 5 winning numbers with this ticket are 0.001845%, i.e. a 1 in 54,201 chance.


### Next Steps
Next steps include:
- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.
- Creating a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. Hint: the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
    - The number of successful outcomes for having four winning numbers exactly
    - The number of successful outcomes for having five winning numbers exactly
    - The number of successful outcomes for having six winning numbers exactly